In [1]:
import numpy as np

In [2]:
#########GENERATE DATA ###################

# number of cities
I = 20

# lambda's, lambda_ii = 0 for each i since no demand arriving that has to move from i to i
rho_i = np.random.uniform(0,1,I)
lambda_ij = np.zeros((I,I))
for i in range(lambda_ij.shape[0]):
    for j in range(lambda_ij.shape[0]):
        if i != j:
            lambda_ij[i,j] = 2 * rho_i[i] * (1-rho_i[j])

# distances assumed integer and symmetric
m_ij = np.zeros((I,I))
for i in range(m_ij.shape[0]):
    random = np.random.randint(100,1500,size=(I,1))
    for j in range(m_ij.shape[0]):
        if i<j:
            m_ij[i,j] = random[j]
            m_ij[j,i] = m_ij[i,j]

# rewards
r_ij = np.zeros((I,I))
for i in range(r_ij.shape[0]):
    for j in range(r_ij.shape[0]):
        r_ij[i,j] = m_ij[i,j] * rho_i[i]
        
# costs
c_ij = np.zeros((I,I))
for i in range(c_ij.shape[0]):
    for j in range(c_ij.shape[0]):
        c_ij[i,j] = 1.2 * m_ij[i,j]



In [3]:
# arrival of the demand Poisson distributed
def demand_simulation():
    D_tij = np.zeros((I,I))
    for i in range(D_tij.shape[0]):
        for j in range(D_tij.shape[0]):
            D_tij[i,j] = np.random.poisson(lambda_ij[i,j]) 
    return D_tij

In [4]:
# state space s=(t,l,d)
# state = np.array([[0],[0],D_tij[0]])

# action set a=(x^L,x^E)
xL = np.zeros((I,I)) 
xE = np.zeros((I,I))

In [5]:
avg_dist = np.zeros(I)
for i in range(0,I):
    avg_dist[i] = np.mean(m_ij[i,:])
    
avg_lambda = np.zeros(I)
for i in range(0,I):
    avg_lambda[i] = np.mean(lambda_ij[i,:])

weighted_distance = np.zeros(I)
for i in range(0,I):
    weighted_distance[i] = np.sum([m_ij[i,j]*lambda_ij[i,j] for j in range(0,I)])/np.sum([lambda_ij[i,j] for j in range(0,I)])

# value function with basis functions:
def v_bar(loc, theta):
  
    return theta[0] + theta[1] * avg_dist[loc] + theta[2] * avg_lambda[loc] + theta[3] * weighted_distance[loc]

def basis_function(loc):
    
    return np.expand_dims(np.array([1, avg_dist[loc], avg_lambda[loc] , weighted_distance[loc]]), axis=1)



In [6]:
# bellman equation for basis functions
def bellmanBasis(pre_s, theta):

    loc = pre_s[1]
    demand = pre_s[2]
    
    # iterate over possible actions
    a = -1
    v = - np.inf
    for j in range(0,I):
        
        if j == loc:
            continue
        
        new_v = 0
        
        if demand[loc,j] == 0:
            new_v = -c_ij[loc,j] + v_bar(j, theta)
        else:
            new_v = r_ij[loc,j] + v_bar(j, theta)
        
        if new_v >= v:
            v = new_v
            a = j
            
    return a, v

In [7]:
def gamma(n, loc, B_prev):
    n = n+1 #because in python we start counting from zero
    
    a = lambda n : 10/(9+n+1)
    
    lambda_n = a(n-1) *(1 - a(n)) /a(n)
    
    f = basis_function(loc)
    
    g = lambda_n + f.T @ B_prev @ f

    return g[0][0]


def B_matrix(n, loc, B_prev, g):
    n = n+1
    
    a = lambda n : 10/(9+n+1)
    
    lambda_n = a(n-1) * (1 - a(n)) /a(n)
    
    f = basis_function(loc)
    
    B = 1/lambda_n*(B_prev - 1/g * (B_prev @ f @ f.T @ B_prev))    
    return B

In [8]:
# Approximate value iteration with basis functions
def app_value_iterBasis(initial_loc, iterations, horizon, initial_theta):
    
    # pre-decision [t, current_loc, demand]
    # post-decision[t, new_loc]
    # pre-decision [t+1, new_loc, new_demand]
        
    theta = initial_theta
    
    v_prev = v_bar(initial_loc, theta)
    B_prev = np.eye(4)* 1e-300
    
    for n in range(0, iterations):
        print('iteration ', n)
        pre_s = [0, initial_loc, demand_simulation()]

        for t in range(0,horizon):
            a, v = bellmanBasis(pre_s, theta)
            
            phi = basis_function(a)
            
            g = gamma(n, a, B_prev)
            
            B = B_matrix(n, a, B_prev, g)
            
            H = 1/g * B_prev
            
            epsilon = v_bar(a, theta) - v
            
            theta = theta - H @ phi * epsilon
            
            pre_s = [t+1, a, demand_simulation()]
            
            B_prev = B_matrix(n, a, B_prev, g)
           
    return theta
    

In [9]:
initial_theta = np.expand_dims(np.random.uniform(0,1,4), axis=1)

theta = app_value_iterBasis(0, 1000, 21,initial_theta)


iteration  0
iteration  1
iteration  2
iteration  3
iteration  4
iteration  5
iteration  6
iteration  7
iteration  8
iteration  9
iteration  10
iteration  11
iteration  12
iteration  13
iteration  14
iteration  15
iteration  16
iteration  17
iteration  18
iteration  19
iteration  20
iteration  21
iteration  22
iteration  23
iteration  24
iteration  25
iteration  26
iteration  27
iteration  28
iteration  29
iteration  30
iteration  31
iteration  32
iteration  33
iteration  34
iteration  35
iteration  36
iteration  37
iteration  38
iteration  39
iteration  40
iteration  41
iteration  42
iteration  43
iteration  44
iteration  45
iteration  46
iteration  47
iteration  48
iteration  49
iteration  50
iteration  51
iteration  52
iteration  53
iteration  54
iteration  55
iteration  56
iteration  57
iteration  58
iteration  59
iteration  60
iteration  61
iteration  62
iteration  63
iteration  64
iteration  65
iteration  66
iteration  67
iteration  68
iteration  69
iteration  70
iteration  71
it

iteration  554
iteration  555
iteration  556
iteration  557
iteration  558
iteration  559
iteration  560
iteration  561
iteration  562
iteration  563
iteration  564
iteration  565
iteration  566
iteration  567
iteration  568
iteration  569
iteration  570
iteration  571
iteration  572
iteration  573
iteration  574
iteration  575
iteration  576
iteration  577
iteration  578
iteration  579
iteration  580
iteration  581
iteration  582
iteration  583
iteration  584
iteration  585
iteration  586
iteration  587
iteration  588
iteration  589
iteration  590
iteration  591
iteration  592
iteration  593
iteration  594
iteration  595
iteration  596
iteration  597
iteration  598
iteration  599
iteration  600
iteration  601
iteration  602
iteration  603
iteration  604
iteration  605
iteration  606
iteration  607
iteration  608
iteration  609
iteration  610
iteration  611
iteration  612
iteration  613
iteration  614
iteration  615
iteration  616
iteration  617
iteration  618
iteration  619
iteration 

In [10]:
print(theta)
print(initial_theta)

[[ 68162.42693138]
 [  -160.10296218]
 [-26551.1177644 ]
 [    88.79446288]]
[[0.96656181]
 [0.16333578]
 [0.12879513]
 [0.33264465]]


In [11]:
def simulationBasis(initial_loc, theta, iterations, horizon):
    
    rewards = np.zeros(iterations)
    for n in range(0, iterations):
        print('iteration ', n)

        loc = initial_loc
        
        for t in range(0,horizon):
            demand = demand_simulation()

            pre_s = [t, loc, demand]

            a, v = bellmanBasis(pre_s, theta)
            
            if demand[loc, a] == 0:
                rewards[n] += -c_ij[loc,a]
            else:
                rewards[n] += r_ij[loc,a]
                
            loc = a

    return rewards


In [12]:
rewardsBasis = simulationBasis(0, theta, 1000, 21)
init_rewardsBasis = simulationBasis(0, initial_theta, 1000, 21)

iteration  0
iteration  1
iteration  2
iteration  3
iteration  4
iteration  5
iteration  6
iteration  7
iteration  8
iteration  9
iteration  10
iteration  11
iteration  12
iteration  13
iteration  14
iteration  15
iteration  16
iteration  17
iteration  18
iteration  19
iteration  20
iteration  21
iteration  22
iteration  23
iteration  24
iteration  25
iteration  26
iteration  27
iteration  28
iteration  29
iteration  30
iteration  31
iteration  32
iteration  33
iteration  34
iteration  35
iteration  36
iteration  37
iteration  38
iteration  39
iteration  40
iteration  41
iteration  42
iteration  43
iteration  44
iteration  45
iteration  46
iteration  47
iteration  48
iteration  49
iteration  50
iteration  51
iteration  52
iteration  53
iteration  54
iteration  55
iteration  56
iteration  57
iteration  58
iteration  59
iteration  60
iteration  61
iteration  62
iteration  63
iteration  64
iteration  65
iteration  66
iteration  67
iteration  68
iteration  69
iteration  70
iteration  71
it

iteration  562
iteration  563
iteration  564
iteration  565
iteration  566
iteration  567
iteration  568
iteration  569
iteration  570
iteration  571
iteration  572
iteration  573
iteration  574
iteration  575
iteration  576
iteration  577
iteration  578
iteration  579
iteration  580
iteration  581
iteration  582
iteration  583
iteration  584
iteration  585
iteration  586
iteration  587
iteration  588
iteration  589
iteration  590
iteration  591
iteration  592
iteration  593
iteration  594
iteration  595
iteration  596
iteration  597
iteration  598
iteration  599
iteration  600
iteration  601
iteration  602
iteration  603
iteration  604
iteration  605
iteration  606
iteration  607
iteration  608
iteration  609
iteration  610
iteration  611
iteration  612
iteration  613
iteration  614
iteration  615
iteration  616
iteration  617
iteration  618
iteration  619
iteration  620
iteration  621
iteration  622
iteration  623
iteration  624
iteration  625
iteration  626
iteration  627
iteration 

iteration  116
iteration  117
iteration  118
iteration  119
iteration  120
iteration  121
iteration  122
iteration  123
iteration  124
iteration  125
iteration  126
iteration  127
iteration  128
iteration  129
iteration  130
iteration  131
iteration  132
iteration  133
iteration  134
iteration  135
iteration  136
iteration  137
iteration  138
iteration  139
iteration  140
iteration  141
iteration  142
iteration  143
iteration  144
iteration  145
iteration  146
iteration  147
iteration  148
iteration  149
iteration  150
iteration  151
iteration  152
iteration  153
iteration  154
iteration  155
iteration  156
iteration  157
iteration  158
iteration  159
iteration  160
iteration  161
iteration  162
iteration  163
iteration  164
iteration  165
iteration  166
iteration  167
iteration  168
iteration  169
iteration  170
iteration  171
iteration  172
iteration  173
iteration  174
iteration  175
iteration  176
iteration  177
iteration  178
iteration  179
iteration  180
iteration  181
iteration 

iteration  672
iteration  673
iteration  674
iteration  675
iteration  676
iteration  677
iteration  678
iteration  679
iteration  680
iteration  681
iteration  682
iteration  683
iteration  684
iteration  685
iteration  686
iteration  687
iteration  688
iteration  689
iteration  690
iteration  691
iteration  692
iteration  693
iteration  694
iteration  695
iteration  696
iteration  697
iteration  698
iteration  699
iteration  700
iteration  701
iteration  702
iteration  703
iteration  704
iteration  705
iteration  706
iteration  707
iteration  708
iteration  709
iteration  710
iteration  711
iteration  712
iteration  713
iteration  714
iteration  715
iteration  716
iteration  717
iteration  718
iteration  719
iteration  720
iteration  721
iteration  722
iteration  723
iteration  724
iteration  725
iteration  726
iteration  727
iteration  728
iteration  729
iteration  730
iteration  731
iteration  732
iteration  733
iteration  734
iteration  735
iteration  736
iteration  737
iteration 

In [15]:
mean = np.mean(rewardsBasis)
std = np.sqrt(np.var(rewardsBasis)) 
initial_meanBasis = np.mean(init_rewardsBasis)

In [16]:
mean

-145.31832688570913

In [17]:
initial_mean

6601.960150231713

In [19]:
np.std(rewardsBasis)

833.8741459277406

In [ ]:
# function that simulates the path followed in one simulation
def simulate_pathBasis(loc, theta):
    path = []
    path.append(loc)
    for t in range(0,21):
        demand = demand_simulation()

        pre_s = [t, loc, demand]

        a, v = bellmanBasis(pre_s, theta)

        loc = a
        path.append(loc)
    
    return path

In [ ]:
simulate_path(0,theta)